# LEET CODE
## Problem-607: Sales Person

Class: Easy

Source: https://leetcode.com/problems/sales-person/

## Description
Table: SalesPerson
| Column Name     | Type    |
|-----------------|---------|
| sales_id        | int     |
| name            | varchar |
| salary          | int     |
| commission_rate | int     |
| hire_date       | date    |

sales_id is the primary key (column with unique values) for this table.

Each row of this table indicates the name and the ID of a salesperson alongside their salary, commission rate, and hire date.

Table: Company
| Column Name | Type    |
|-------------|---------|
| com_id      | int     |
| name        | varchar |
| city        | varchar |

com_id is the primary key (column with unique values) for this table.

Each row of this table indicates the name and the ID of a company and the city in which the company is located.

Table: Orders
| Column Name | Type |
|-------------|------|
| order_id    | int  |
| order_date  | date |
| com_id      | int  |
| sales_id    | int  |
| amount      | int  |

order_id is the primary key (column with unique values) for this table.

com_id is a foreign key (reference column) to com_id from the Company table.

sales_id is a foreign key (reference column) to sales_id from the SalesPerson table.

Each row of this table contains information about one order. This includes the ID of the company, the ID of the salesperson, the date of the order, and the amount paid.

## To Do
Write a solution to find the names of all the salespersons who did not have any orders related to the company with the name "RED".

Return the result table in any order.

The result format is in the following example.

## Example
Input: 

SalesPerson table:
| sales_id | name | salary | commission_rate | hire_date  |
|----------|------|--------|-----------------|------------|
| 1        | John | 100000 | 6               | 4/1/2006   |
| 2        | Amy  | 12000  | 5               | 5/1/2010   |
| 3        | Mark | 65000  | 12              | 12/25/2008 |
| 4        | Pam  | 25000  | 25              | 1/1/2005   |
| 5        | Alex | 5000   | 10              | 2/3/2007   |

Company table:
| com_id | name   | city     |
|--------|--------|----------|
| 1      | RED    | Boston   |
| 2      | ORANGE | New York |
| 3      | YELLOW | Boston   |
| 4      | GREEN  | Austin   |

Orders table:
| order_id | order_date | com_id | sales_id | amount |
|----------|------------|--------|----------|--------|
| 1        | 1/1/2014   | 3      | 4        | 10000  |
| 2        | 2/1/2014   | 4      | 5        | 5000   |
| 3        | 3/1/2014   | 1      | 1        | 50000  |
| 4        | 4/1/2014   | 1      | 4        | 25000  |

Output: 
| name |
|------|
| Amy  |
| Mark |
| Alex |

Explanation: 
- According to orders 3 and 4 in the Orders table, it is easy to tell that only salesperson John and Pam have sales to company RED, so we report all the other names in the table salesperson.


## Importing

In [1]:
import pandas as pd

## Define Schema

In [4]:
data = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
SalesPerson = pd.DataFrame(data, columns=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']).astype({'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'})
data = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
Company = pd.DataFrame(data, columns=['com_id', 'name', 'city']).astype({'com_id':'Int64', 'name':'object', 'city':'object'})
data = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
Orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'})

In [5]:
display(SalesPerson)
display(Company)
display(Orders)

,sales_id,name,salary,commission_rate,hire_date
0,1,John,100000,6,2006-04-01
1,2,Amy,12000,5,2010-05-01
2,3,Mark,65000,12,2008-12-25
3,4,Pam,25000,25,2005-01-01
4,5,Alex,5000,10,2007-02-03


,com_id,name,city
0,1,RED,Boston
1,2,ORANGE,New York
2,3,YELLOW,Boston
3,4,GREEN,Austin


,order_id,order_date,com_id,sales_id,amount
0,1,2014-01-01,3,4,10000
1,2,2014-02-01,4,5,5000
2,3,2014-03-01,1,1,50000
3,4,2014-04-01,1,4,25000


## Task

In [7]:
## Define SQL Query
query = """
SELECT name 
FROM SalesPerson
WHERE sales_id NOT IN (
    SELECT sales_id
    FROM Orders
    LEFT JOIN Company USING(com_id)
    WHERE name = 'RED'
)
;
"""
# Excute the query using pandasql
result = sqldf(query, env={
    'SalesPerson':SalesPerson,
    'Company': Company,
    'Orders': Orders
    })

# Display the result dataframe
display(result)

,name
0,Amy
1,Mark
2,Alex


### Improved runtime

In [9]:
## Define SQL Query
query = """
 SELECT name FROM SalesPerson 
	 WHERE sales_id NOT IN(
		 SELECT sales_id FROM Orders WHERE com_id IN (
				 SELECT com_id FROM Company WHERE name="RED" )
	 );
"""
# Excute the query using pandasql
result = sqldf(query, env={
    'SalesPerson':SalesPerson,
    'Company': Company,
    'Orders': Orders
    })

# Display the result dataframe
display(result)

,name
0,Amy
1,Mark
2,Alex


In [31]:
def sales_person(sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    red_company = company[company.name=='RED']['com_id'].values

    red_orders = orders[orders.com_id.isin(red_company)]['sales_id'].values.tolist()

    return sales_person[~sales_person.sales_id.isin(red_orders)][['name']]


In [32]:
sales_person(SalesPerson, Company, Orders)

,name
1,Amy
2,Mark
4,Alex
